In [15]:
import pandas as pd
# no cut off the output
pd.set_option('display.max_rows', None)

chars = pd.read_csv('../词汇.csv')
chars.head()

,No.,级别,词语,拼音,词性
0,1,高等,阿拉伯语,Ālābóyǔ,名
1,2,四级,阿姨,āyí,名
2,3,四级,啊,ā,叹
3,4,二级,啊,a,助
4,5,高等,哎,āi,叹


In [20]:

# Define the order
level_order = ['一级', '二级', '三级', '四级', '五级', '六级', '高等', 'N/A']

# Apply the order
chars['级别'] = pd.Categorical(chars['级别'], level_order)

# Sort the dataframe
chars.sort_values('级别', inplace=True)

# filter 词语 column with less than 2 characters
chars = chars[chars['词语'].str.len() > 1]

# Group by level and count and more than two characters
chars.groupby('级别').count()


/tmp/ipykernel_56194/2522267770.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  chars.groupby('级别').count()


,No.,词语,拼音,词性
级别,,,,
一级,295,295,295,219
二级,593,593,593,498
三级,813,813,813,752
四级,831,831,831,754
五级,896,896,896,820
六级,948,948,948,853
高等,5146,5146,5146,4268
N/A,0,0,0,0


In [ ]:
import pandas as pd
import unicodedata
# open harry.txt
with open('text_to_analyse.txt', 'r') as f:
    text = f.read()

# count characters
from collections import Counter
counts = Counter(text)

# convert to dataframe
counts = pd.DataFrame.from_dict(counts, orient='index').reset_index()
counts.columns = ['character', 'count']

counts = counts.sort_values(by='count', ascending=False)

counts.head()

FileNotFoundError: [Errno 2] No such file or directory: 'text_to_analyse.txt'

In [ ]:
import unicodedata

def is_han(char):
    try:
        return all('CJK UNIFIED' in unicodedata.name(c) for c in char)
    except:
        return False

# filter out non-Chinese characters
counts['is_chinese'] = counts['character'].apply(is_han)
counts = counts[counts['is_chinese'] == True]

In [ ]:

def filter_by_level(level):
    return counts[counts['character'].isin(chars[chars['level'] == level]['character'])].sort_values('count', ascending=False)


# '一级', '二级', '三级', '四级', '五级', '六级', '高等'
filter_by_level('高等').head(10)

,character,count,is_chinese
308,着,16641,True
258,和,7761,True
108,斯,7730,True
9,罗,6545,True
1600,魔,5398,True
1980,赫,3416,True
3121,邓,2411,True
1300,杖,1912,True
761,伯,1503,True
234,别,1493,True


In [ ]:
# print the chars that are not in the list of chars

non_in_hsk = counts[~counts['character'].isin(
    chars['character'])].sort_values('count', ascending=False)
non_in_hsk.head(10)

,character,count,is_chinese
958,莱,1637,True
847,弗,1620,True
682,妮,1260,True
1645,咒,1180,True
1651,韦,890,True
786,娜,825,True
3277,卢,723,True
862,莉,446,True
1767,帚,416,True
679,嗯,381,True


In [ ]:
def get_level(character):
    try:
        return chars[chars['character'] == character]['level'].values[0]
    except:
        return 'N/A'


total_count = counts['count'].sum()

counts = counts.copy()  # copy the dataframe
counts['percentage'] = counts['count'] / total_count * 100
counts['level'] = counts['character'].apply(get_level)

counts['level'] = counts['character'].apply(get_level)
counts.groupby('level').sum()['percentage'].sort_values(ascending=False)

level
一级     50.670184
二级     17.260424
三级     11.426207
高等      6.683129
四级      5.978635
五级      3.309374
六级      3.240342
N/A     1.431704
Name: percentage, dtype: float64

In [ ]:
# Define the order
level_order = ['一级', '二级', '三级', '四级', '五级', '六级', '高等', 'N/A']

# Convert the 'level' column to a categorical type with the specified order
counts['level'] = pd.Categorical(counts['level'], categories=level_order, ordered=True)

# Groupby 'level' and sum the 'percentage' column
counts = counts.groupby('level').sum()['percentage'].reset_index()
# cummulatively sum the percentage
counts['cum_percentage'] = counts['percentage'].cumsum()

# save to csv
counts.to_csv('level_percentage.csv', index=False)

/tmp/ipykernel_1379/1427996165.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  counts = counts.groupby('level').sum()['percentage'].reset_index()


In [ ]:
counts

,level,percentage,cum_percentage
0,一级,50.670184,50.670184
1,二级,17.260424,67.930609
2,三级,11.426207,79.356816
3,四级,5.978635,85.335451
4,五级,3.309374,88.644825
5,六级,3.240342,91.885167
6,高等,6.683129,98.568296
7,N/A,1.431704,100.000000
